In [3]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from sklearn.metrics import mean_squared_error

In [5]:
from prophet import Prophet

In [6]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 12)

In [7]:
import warnings
warnings.simplefilter('ignore')

In [8]:
from multi_prophet import MultiProphet

In [9]:
def relative_rmse(target, preds):
    rmse = mean_squared_error(target, preds, squared=False)
    avg = np.full(len(preds), preds.mean())
    const_rmse = mean_squared_error(target, avg, squared=False)
    return rmse / const_rmse

In [10]:
weather_df = pd.read_csv('NASA_weather_latitude_51.5_longitude_37.0.csv', skiprows=8)
weather_df["DATE"] = pd.to_datetime(weather_df["DAY"], format='%Y%m%d')

In [11]:
train_tmp = weather_df[weather_df.DATE.dt.year > 1999].copy()
train_data = train_tmp[train_tmp.DATE.dt.year < 2015].copy()

In [12]:
test_tmp = weather_df[weather_df.DATE.dt.year > 2014].copy()
test_data = test_tmp[test_tmp.DATE.dt.year < 2020].copy()

In [13]:
display(train_data.head())
display(test_data.tail())

,DAY,IRRAD,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH,DATE
5844,20000101,3550.0,-12.80,-7.20,0.276993,1.56,1.21,NaN,2000-01-01
5845,20000102,4140.0,-12.74,-6.73,0.297244,3.87,2.47,NaN,2000-01-02
5846,20000103,4590.0,-16.81,-11.44,0.185930,1.34,0.40,NaN,2000-01-03
5847,20000104,4600.0,-15.39,-8.39,0.220140,2.95,0.12,NaN,2000-01-04
5848,20000105,3610.0,-11.50,-5.81,0.310407,3.70,0.65,NaN,2000-01-05


,DAY,IRRAD,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH,DATE
13144,20191227,1280.0,-1.88,0.75,0.547656,1.42,1.23,NaN,2019-12-27
13145,20191228,990.0,-3.79,-1.13,0.423742,3.27,1.59,NaN,2019-12-28
13146,20191229,1070.0,-6.74,-3.27,0.356630,2.94,1.63,NaN,2019-12-29
13147,20191230,750.0,-6.81,-3.80,0.363870,3.76,0.47,NaN,2019-12-30
13148,20191231,1200.0,-3.92,-0.81,0.474842,6.01,2.31,NaN,2019-12-31


In [14]:
train_data.reset_index(drop=True)
train_data = train_data.rename(columns={"DATE": "ds"})
train_data.reset_index(drop=True)
train_data = train_data.rename(columns={"DATE": "ds"})

test_data.reset_index(drop=True)
test_data = test_data.rename(columns={"DATE": "ds"})
test_data.reset_index(drop=True)
test_data = test_data.rename(columns={"DATE": "ds"})

In [15]:
m = MultiProphet(columns=["IRRAD", "TMIN", "TMAX", "VAP", "WIND", "RAIN"])
m.fit(train_data)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [16]:
future = m.make_future_dataframe(periods=1826, include_history=False)
forecast = m.predict(future)

In [23]:
forecast['IRRAD'].head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2015-01-01,12540.741661,-1832.858297,8201.395868,12540.741661,12540.741661,-9582.666223,-9582.666223,-9582.666223,43.364291,43.364291,43.364291,-9626.030514,-9626.030514,-9626.030514,0.0,0.0,0.0,2958.075438
1,2015-01-02,12540.908193,-1959.434463,8425.593128,12540.908193,12540.908193,-9448.082672,-9448.082672,-9448.082672,100.042869,100.042869,100.042869,-9548.125541,-9548.125541,-9548.125541,0.0,0.0,0.0,3092.825521
2,2015-01-03,12541.074726,-1838.050665,8488.045317,12541.074726,12541.074726,-9393.163782,-9393.163782,-9393.163782,72.239966,72.239966,72.239966,-9465.403748,-9465.403748,-9465.403748,0.0,0.0,0.0,3147.910945
3,2015-01-04,12541.241259,-1845.511512,8292.318836,12541.241259,12541.241259,-9315.905051,-9315.905051,-9315.905051,62.933702,62.933702,62.933702,-9378.838753,-9378.838753,-9378.838753,0.0,0.0,0.0,3225.336209
4,2015-01-05,12541.407792,-2187.258281,8395.585032,12541.407792,12541.407792,-9400.257199,-9400.257199,-9400.257199,-110.818565,-110.818565,-110.818565,-9289.438633,-9289.438633,-9289.438633,0.0,0.0,0.0,3141.150593


In [24]:
test_data['_IRRAD'] = forecast['IRRAD'].yhat.to_numpy()

In [30]:
test_data['_IRRAD'] = forecast['IRRAD'].yhat.to_numpy()
test_data['_TMIN'] = forecast['TMIN'].yhat.to_numpy()
test_data['_TMIN'] = forecast['TMIN'].yhat.to_numpy()
test_data['_TMAX'] = forecast['TMAX'].yhat.to_numpy()
test_data['_VAP'] = forecast['VAP'].yhat.to_numpy()
test_data['_WIND'] = forecast['WIND'].yhat.to_numpy()
test_data['_RAIN'] = forecast['RAIN'].yhat.to_numpy()

In [31]:
test_data.head()

,DAY,IRRAD,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH,ds,_IRRAD,_TMIN,_TMAX,_VAP,_WIND,_RAIN
11323,20150101,1570.0,-13.97,-5.31,0.296775,4.51,0.68,NaN,2015-01-01,2958.075438,-10.631710,-4.388340,0.303586,3.606764,1.546504
11324,20150102,1510.0,-5.18,-0.77,0.492191,5.11,0.96,NaN,2015-01-02,3092.825521,-10.735548,-4.482948,0.295376,3.572931,1.295587
11325,20150103,2320.0,-2.38,0.68,0.571513,6.33,1.14,NaN,2015-01-03,3147.910945,-10.849946,-4.411090,0.302538,3.529855,1.523292
11326,20150104,1530.0,-4.93,-0.16,0.506322,5.75,0.59,NaN,2015-01-04,3225.336209,-10.943394,-4.502300,0.301302,3.500323,1.490125
11327,20150105,1800.0,-11.96,-2.27,0.369812,3.33,1.33,NaN,2015-01-05,3141.150593,-11.026453,-4.511247,0.301002,3.475727,1.472167


In [32]:
output = test_data[['DAY', '_IRRAD', '_TMIN', '_TMAX', '_VAP', '_WIND', '_RAIN', 'SNOWDEPTH']].copy()

In [33]:
output = output.rename(columns={
    '_IRRAD': 'IRRAD', 
    '_TMIN': 'TMIN', 
    '_TMAX': 'TMAX', 
    '_VAP': 'VAP', 
    '_WIND': 'WIND', 
    '_RAIN': 'RAIN' 
    })

In [34]:
output.head()

,DAY,IRRAD,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH
11323,20150101,2958.075438,-10.631710,-4.388340,0.303586,3.606764,1.546504,NaN
11324,20150102,3092.825521,-10.735548,-4.482948,0.295376,3.572931,1.295587,NaN
11325,20150103,3147.910945,-10.849946,-4.411090,0.302538,3.529855,1.523292,NaN
11326,20150104,3225.336209,-10.943394,-4.502300,0.301302,3.500323,1.490125,NaN
11327,20150105,3141.150593,-11.026453,-4.511247,0.301002,3.475727,1.472167,NaN


In [35]:
output.to_csv('multi_prophet-wofost.csv', index=False)

# MONICA

In [36]:
weather_df = pd.read_csv('monica_weather.csv', delimiter=';')

In [37]:
weather_df["de-date"] = pd.to_datetime(weather_df["de-date"], format='%d.%m.%Y')

In [38]:
train_tmp = weather_df[weather_df['de-date'].dt.year > 1999].copy()
train_data = train_tmp[train_tmp['de-date'].dt.year < 2015].copy()

test_tmp = weather_df[weather_df['de-date'].dt.year > 2014].copy()
test_data = test_tmp[test_tmp['de-date'].dt.year < 2020].copy()

In [39]:
display(train_data.head())
display(test_data.tail())

,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid
0,2000-01-01,-8.74,-10.44,-5.97,1.41,3.55,0.51,98.06
1,2000-01-02,-8.92,-12.29,-6.56,3.52,4.14,2.47,99.19
2,2000-01-03,-14.60,-17.18,-11.20,1.44,4.59,0.41,99.88
3,2000-01-04,-12.80,-16.44,-8.52,3.33,4.60,0.11,98.88
4,2000-01-05,-8.49,-12.60,-5.84,3.98,3.61,0.73,98.62


,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid
7300,2019-12-27,-1.74,-2.65,-0.12,1.86,1.28,1.65,88.50
7301,2019-12-28,-3.89,-4.99,-2.65,3.00,0.99,0.64,85.12
7302,2019-12-29,-5.73,-7.42,-4.26,2.49,1.07,0.39,89.38
7303,2019-12-30,-5.68,-7.36,-4.02,3.86,0.75,0.83,92.12
7304,2019-12-31,-2.42,-4.20,-1.51,5.96,1.20,3.03,93.25


In [40]:
train_data.reset_index(drop=True)
train_data = train_data.rename(columns={"de-date": "ds"})

test_data.reset_index(drop=True)
test_data = test_data.rename(columns={"de-date": "ds"})

In [41]:
test_data.head()

,ds,tavg,tmin,tmax,wind,globrad,precip,relhumid
5479,2015-01-01,-8.40,-13.63,-4.55,4.55,1.57,1.00,96.81
5480,2015-01-02,-2.18,-4.57,-0.70,5.08,1.51,1.27,97.00
5481,2015-01-03,-0.37,-2.44,0.62,6.26,2.32,1.20,96.25
5482,2015-01-04,-1.85,-4.76,-0.26,6.07,1.53,0.58,95.75
5483,2015-01-05,-5.77,-12.23,-2.49,3.46,1.80,1.21,95.56


In [43]:
m = MultiProphet(columns=["tavg", "tmin", "tmax", "wind", "globrad", "precip", "relhumid"])
m.fit(train_data)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [44]:
future = m.make_future_dataframe(periods=1826, include_history=False)
forecast = m.predict(future)

In [45]:
test_data['_tavg'] = forecast['tavg'].yhat.to_numpy()
test_data['_tmin'] = forecast['tmin'].yhat.to_numpy()
test_data['_tmax'] = forecast['tmax'].yhat.to_numpy()
test_data['_wind'] = forecast['wind'].yhat.to_numpy()
test_data['_globrad'] = forecast['globrad'].yhat.to_numpy()
test_data['_precip'] = forecast['precip'].yhat.to_numpy()
test_data['_relhumid'] = forecast['relhumid'].yhat.to_numpy()

In [46]:
test_data.head()

,ds,tavg,tmin,tmax,wind,globrad,precip,relhumid,_tavg,_tmin,_tmax,_wind,_globrad,_precip,_relhumid
5479,2015-01-01,-8.40,-13.63,-4.55,4.55,1.57,1.00,96.81,-7.820746,-11.103810,-4.648889,3.574997,2.958923,1.550664,90.228959
5480,2015-01-02,-2.18,-4.57,-0.70,5.08,1.51,1.27,97.00,-7.911224,-11.201153,-4.753649,3.532126,3.093674,1.315943,90.220391
5481,2015-01-03,-0.37,-2.44,0.62,6.26,2.32,1.20,96.25,-7.868663,-11.275977,-4.684840,3.498026,3.148759,1.462944,90.381198
5482,2015-01-04,-1.85,-4.76,-0.26,6.07,1.53,0.58,95.75,-7.904355,-11.316832,-4.753876,3.465184,3.226185,1.471841,90.247751
5483,2015-01-05,-5.77,-12.23,-2.49,3.46,1.80,1.21,95.56,-8.023031,-11.433279,-4.747446,3.442736,3.142000,1.378593,90.433939


In [47]:
output = test_data[['ds', '_tavg', '_tmin', '_tmax', '_wind', '_globrad', '_precip', '_relhumid']].copy()

In [48]:
output = output.rename(columns={
    'ds': 'de-date',
    '_tavg': 'tavg', 
    '_tmin': 'tmin', 
    '_tmax': 'tmax', 
    '_wind': 'wind', 
    '_globrad': 'globrad', 
    '_precip': 'precip',
    '_relhumid': 'relhumid'
    })

In [49]:
output.head()

,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid
5479,2015-01-01,-7.820746,-11.103810,-4.648889,3.574997,2.958923,1.550664,90.228959
5480,2015-01-02,-7.911224,-11.201153,-4.753649,3.532126,3.093674,1.315943,90.220391
5481,2015-01-03,-7.868663,-11.275977,-4.684840,3.498026,3.148759,1.462944,90.381198
5482,2015-01-04,-7.904355,-11.316832,-4.753876,3.465184,3.226185,1.471841,90.247751
5483,2015-01-05,-8.023031,-11.433279,-4.747446,3.442736,3.142000,1.378593,90.433939


In [50]:
output.to_csv('multi_prophet-monica.csv', index=False)